In [1]:
import numpy as np
import tables
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/eframe/dmi/src')
import calibrate as calib
import eventAnalysis as ea
import compton
import sys
import pandas as pd
import more_itertools as mit

In [2]:
def subtract_baseline(rdata, sample_window=15):
    """ Usage: subtract_baseline(rdata, sample_window=30) return rdata with the
    mean of the first sample_window samples having been subtracted from the
    entire signal
    """
    return rdata - rdata[:, 0:sample_window].mean(axis=1)[:, np.newaxis]

In [3]:
def search_t50(signal):
    """ Returns the t50 value of a raw signal
    """
    tMin = np.argmax(signal)
    sigMax = np.mean( signal[tMin:] )

    # Loop over signal to find t50 point
    t = 0
    t50_region_end = len(signal);
    while t < t50_region_end:
        if signal[t] > ( sigMax / 2 ):
            break
        t = t + 1

    #  Calculate t50 with linear interp
    t2 = t
    t1 = t2 - 1
    s2 = signal[t2]
    s1 = signal[t1]
    t50 = ( ( t2 - t1 ) / ( s2 - s1) ) * ( sigMax / 2 - s1 ) + t1

    return t50

In [4]:
def lump_edata(edata, twindow=30):
    """ Finding lower and upper indices of events that lie within the specified time window
    """
    t = np.diff( edata['timestamp'] )
    tmask = ( t > 25 )
    T = np.ma.masked_array( t, tmask ).filled( 0 )
    i2 = np.where( T == 0 )[0] + 1
    i1 = np.insert( i2[ :-1 ], 0, 0 )  

    return i1, i2

In [5]:
def is_single(event):
    """ Determines whether an event is a singles event in the first detector. Allows for charge sharing.
    """
    mask = event['trigger'] == 1
    fired = event[mask]

    d1 = ea.inge1(fired)
    d1_AC = ea.onAC(d1)
    d1_DC = ea.onDC(d1)

    lensAC = len(d1_AC)
    lensDC = len(d1_DC)

    e1_AC = np.sum( d1_AC['ADC_value'] )
    e1_DC = np.sum( d1_DC['ADC_value'] )

    if ( lensAC == 1 ) and ( lensDC  == 1 ):
        match = ea.checkForEnergyMatch( e1_AC, e1_DC, sigma = 2 )
        if np.sum(match) == 1:
            return True
        else: return False
    else:
        return False

In [8]:
# Time-correlating event data
file = '/Users/eframe/dissertation/data/cs137_raw.h5'
f = tables.open_file( file, 'r' )
edata = f.root.EventData.read().view( np.recarray )
edata.rid = np.arange( len( edata ) )
edata = np.sort( edata, order='timestamp' )
i1, i2 = lump_edata( edata, twindow=25 )

In [9]:
# Subtracting baseline from raw data
rdata = f.root.RawData.read().view( np.recarray )
rdata2 = subtract_baseline( rdata )

In [11]:
single = []
for i in np.arange( 0, 200 ):
    event = edata[i1[i]:i2[i]]
    if is_single( event ) :
        single.append(event)

In [12]:
pos = []
for i in np.arange(len(single) ): 
    event = single[i]
    iterable = np.sort(event, order='detector')
    strips = [list(group) for group in mit.consecutive_groups(iterable.detector)]
    
    # Getting indices of sorted strips ##
    yindex = strips[0] - strips[0][0] 
    xindex = strips[1] - strips[1][0] + yindex[-1] + 1 
    
    masktrigy = iterable[yindex].trigger == 1
    masktrigx = iterable[xindex].trigger == 1
    
    ystrip = np.sort( iterable[yindex], order='ADC_value' ).detector
    ypos = ( ystrip[-1] - 38 * 0 ) * 2 - 1 
     
    xstrip = np.sort( iterable[xindex], order='ADC_value' ).detector
    xpos = ( xstrip[-1] - 38 * 2 ) * 2 - 1 
       
    # Getting interpolated z-coordinate
    esort = np.sort( event, order='ADC_value' )
    detsort = np.sort( esort[-2:], order='detector' )
    E = max( esort.ADC_value )
    
    trigsort = np.sort( np.sort( event, order='trigger' )[-2:], order = 'detector' )
    rid1 = trigsort['rid'][0]
    rid2 = trigsort['rid'][1]
    raw1 = rdata2[rid1]
    raw2 = rdata2[rid2]
    
    t501 = search_t50(raw1)
    t502 = search_t50(raw2)
    
    dt50 = t502 - t501
    dt = int( detsort.timestamp[-1] ) - int( detsort.timestamp[0] )
    dt50_new = dt + dt50
    zpos = dt50_new * 0.3215503 + 7.9596899
    
    ystrip = np.sort(iterable[yindex], order='ADC_value').detector
    yold = ( ystrip[-1] - 38*0 ) * 2 - 1 
     
    xstrip = np.sort(iterable[xindex], order='ADC_value').detector
    xold = ( xstrip[-1] - 38*2 ) * 2 - 1 
             
    pos.append([ E, xpos, ypos, zpos, dt, dt50 ] )

In [13]:
l11 = f.root.Interactions.Single.read()
mask = l11['det'] == 1
l11[mask][:30]

array([( 56.78905 , 69., 29.,  0.8855834 , -22.,  0.        ,  6.303787  , 1, False),
       (139.4592  ,  5., 47., -0.03346825, -24., -0.8581886 ,  0.28779602, 1,  True),
       (180.63846 , 67., 61.,  1.6393957 , -18., -1.655693  ,  0.20211792, 1,  True),
       (262.92093 , 31., 55.,  2.4688945 , -14., -3.0760078 ,  2.1054382 , 1,  True),
       (233.22505 , 35., 39.,  0.5020571 , -21., -2.1927414 ,  4.0598907 , 1,  True),
       (168.08696 , 19., 15.,  9.073249  ,   4., -0.5369072 ,  8.946686  , 1,  True),
       (159.92213 , 31.,  9.,  5.8910103 ,  -6., -0.43345642,  6.0513153 , 1,  True),
       (228.63524 , 39., 13.,  4.904349  ,  -8., -1.5019073 ,  0.05288696, 1,  True),
       (120.99294 ,  9.,  9.,  9.07346   ,   3.,  0.46374893,  3.7119904 , 1,  True),
       (214.79497 , 45., 67., 11.920489  ,  11.,  1.3178215 ,  3.441452  , 1,  True),
       (226.86273 , 31., 25.,  8.187056  ,   1., -0.2929077 ,  0.4406433 , 1,  True),
       (420.6143  , 13., 15.,  0.5747366 , -17., -5.96

In [14]:
pos

[[56.78905, 69, 29, 0.7385080811480931, -22, -0.4573941273011002],
 [139.4592, 5, 47, -0.03329498614558002, -24, -0.8576502218955468],
 [180.63846, 67, 61, 1.6417267098142396, -18, -1.6484443963689657],
 [262.92093, 31, 55, 2.4829758269222877, -14, -3.0322157157922476],
 [168.08696, 19, 15, 9.089228724286516, 4, -0.48720954610673317],
 [0.9284982, 1, 15, 20.85930041858603, 1, 39.116928886665725],
 [120.99294, 9, 9, 9.164713928222117, 3, 0.7475444066515138],
 [16.774776, -1, 13, 20.3993469768575, 2, 36.68650434117929],
 [62.36355, 53, 13, 6.167886220454308, -4, -1.5723900103520094]]